## 빅데이터 실습
### 데이터포털 OpenAPI 활용
> 1. 공공데이터포털에 있는 한국관광공사_관광빅데이터 OpenAPI 활용
>> - 기본 URL : http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList
>>     - _https 프로토콜로 서비스 요청을 하면 SSL 에러가 뜬다._ 
>> - 쿼리파라미터
>>     - `serviceKey` = 본인 서비스 키
>>     - `numOfRows` = 한 페이지에 나타낼 결과수(10)
>>     - `pageNo` = 페이지번호
>>     - `MobileOS` = IOS , AND , WIN , ETC(기타)
>>     - `startYmd` = 시작일자
>>     - `endYmd` = 종료일자
>>     - `MobileApp` = AppTest

In [1]:
# 필수 라이브러리 등록
import requests
import json
import xmltodict
import pandas as pd

#### 추가 모듈
- XML 을 딕셔너리로 변환해주는 모듈 : `xmltodict`
- 명령어 : `pip3 install xmltodict`

In [2]:
!pip3 install xmltodict

In [3]:
# Open API 서비스키 
apiKey = r'API KEY'

# 기준날자
ymd = '20231201'

In [4]:
# 필요 URL 생성
baseUrl = 'http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?'
param = 'numOfRows=60&pageNo=1&MobileOS=ETC&MobileApp=AppTest&' + \
        f'serviceKey={apiKey}&startYmd={ymd}&endYmd={ymd}'

totalUrl = baseUrl + param
print(totalUrl)

http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?numOfRows=60&pageNo=1&MobileOS=ETC&MobileApp=AppTest&serviceKey=%2FBKGfux%2Bde%2Bi0jFLG%2BmFKccXFkvro4xnwk9SpIptRUr6KXmn7wNn7FJCquD4VzMZKSuR%2BjoNcP9jo%2FxzTJNs9A%3D%3D&startYmd=20231201&endYmd=20231201


#### 함수 생성

In [5]:
# 웹으로 OpenAPI 호출
def getRequestUrl(url):
    try:
        res = requests.get(url, verify=False)
        if res.status_code == 200:
            return res.text
    except Exception as e:
        print(f'getRequestUrl() 호출 예외 : {e}')
        return None

In [6]:
# OpenAPI 호출 함수
def getTouristDataService(ymd):
    apiKey = r'API KEY'

    baseUrl = 'http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?'
    param = 'numOfRows=60&pageNo=1&MobileOS=ETC&MobileApp=AppTest&' + \
            f'serviceKey={apiKey}&startYmd={ymd}&endYmd={ymd}'
    totalUrl = baseUrl + param
    result = getRequestUrl(totalUrl)

    if result == None:
        return None
    else:
        return result

In [7]:
# 실행테스트
xmlData = getTouristDataService('20231202')


In [8]:
# XML 데이터를 Dict 로 변환
dictData = xmltodict.parse(xmlData)

In [9]:
# 추가 라이브러리 등록
from tqdm import tqdm
import time

In [10]:
# 반복문으로 처리
totalList = []
year = 2023
month = 12

for day in tqdm(range(1, 32)):  # 1일부터 31일까지 반복
    reqYmd = f'{year}{month}{day:02d}'
    data = getTouristDataService(reqYmd)
    if xmltodict.parse(data)['response']['body']['items'] == None:
        resList = []
    else:
        resList = xmltodict.parse(data)['response']['body']['items']['item']
    totalList = totalList + resList


100%|██████████| 31/31 [01:42<00:00,  3.31s/it]


In [11]:
len(totalList)

1530

In [12]:
dfTourist = pd.DataFrame(totalList)

In [13]:
dfTourist.tail()

,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
1525,51,강원특별자치도,7,일요일,2,외지인(b),694498.0,20231231
1526,51,강원특별자치도,7,일요일,3,외국인(c),2551.5179999999996,20231231
1527,52,전북특별자치도,7,일요일,1,현지인(a),910780.5,20231231
1528,52,전북특별자치도,7,일요일,2,외지인(b),408299.5,20231231
1529,52,전북특별자치도,7,일요일,3,외국인(c),1043.7619000000004,20231231


In [14]:
# 결측치가 없다
dfTourist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1530 entries, 0 to 1529
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   areaCode    1530 non-null   object
 1   areaNm      1530 non-null   object
 2   daywkDivCd  1530 non-null   object
 3   daywkDivNm  1530 non-null   object
 4   touDivCd    1530 non-null   object
 5   touDivNm    1530 non-null   object
 6   touNum      1530 non-null   object
 7   baseYmd     1530 non-null   object
dtypes: object(8)
memory usage: 95.8+ KB


In [15]:
# 엑셀로 저장
dfTourist.to_excel('./data/Tourist_datapotal.xlsx')